In [2]:
import numpy as np
import pandas as pd

# Importing data

### Becelona Data

In [24]:
barcelona_calendar = pd.read_csv("barcelona_calendar.csv")
barcelona_listings = pd.read_csv("barcelona_listings.csv")
barcelona_reviews = pd.read_csv("barcelona_reviews.csv")

### Berlin Data

In [25]:
berlin_calendar = pd.read_csv("berlin_calendar.csv")
berlin_listings = pd.read_csv("berlin_listings_filtered.csv")
berlin_reviews = pd.read_csv("berlin_reviews.csv")

### Madrid Data

In [26]:
madrid_calendar = pd.read_csv("madrid_calendar.csv")
madrid_listings = pd.read_csv("madrid_listings_filtered.csv")
madrid_reviews = pd.read_csv("madrid_reviews.csv")

# Data management
## amenities
### amenities: AMENITY; AMENITY_ID

In [57]:
all_amenities = pd.concat([berlin_listings['amenities'], madrid_listings['amenities'], barcelona_listings['amenities']]) #concatenate all amenities
amenities = all_amenities.str.replace('{','').str.replace('}','').str.replace('"', '') #remove '{' , '}' and '"' 
amenities = amenities.str.split(',', expand = True) #seperate all amenities
a = amenities.sort_index().values #change into a np array
a = a.ravel() #ravel into a 1D array
a = a[a != None] #removes all None
a = np.unique(a) #same as drop_duplicates
a = a[1:] #the first one was an empty string
amenities = pd.Series(a) #to pd
amenities = amenities.sort_values() # those are the amenities sorted
#note: the toilet amenity has a space at the beginning

In [58]:
amenities = pd.DataFrame(amenities).rename(index = str, columns = {0 : 'AMENITY'}) #gives amenities column a name
amenities['AMENITY_ID'] = range(amenities['AMENITY'].count()) #create indexes for each amenity
amenities= amenities.set_index('AMENITY_ID') #defines the created indexes as the index

In [59]:
amenities.head(5)

,AMENITY
AMENITY_ID,
0,toilet
1,24-hour check-in
2,Accessible-height bed
3,Accessible-height toilet
4,Air conditioning


In [60]:
amenities.to_csv('amenities.csv')

### amenities_AND_listings: AMENITY_ID; LISTING_ID

In [8]:
am_lis = all_amenities.str.replace('{','').str.replace('}','').str.replace('"','') 
am_lis = am_lis.map(lambda x: np.array(x.split(',')))
am_lis = am_lis.map(lambda l : [amenities[amenities['AMENITY'] == a].index[0] for a in l if len(l)>1]) #len(l)>1 is because empty amenities list still have a length of 1 for some reason

In [10]:
am_lis = pd.DataFrame(am_lis).rename(index = str, columns = {'amenities' : 'AMENITY_ID'})
am_lis['LISTING_ID'] = berlin_listings['id'].values

In [11]:
am_lis_np = am_lis.values
temp = np.array([])
for x in am_lis_np:
    for i in x[0]:
        temp = np.insert(temp, 0, np.array([i, x[1]]))
#would be great to be able to do it without a for...

In [21]:
temp = np.resize(temp, (temp.size, 2)).astype('int')
am_lis = pd.DataFrame({'AMENITY_ID' : temp[:,0], 'LISTING_ID' : temp[:,1]})
am_lis.head()

,AMENITY_ID,LISTING_ID
0,78,2173438
1,62,2173438
2,64,2173438
3,44,2173438
4,40,2173438


In [22]:
am_lis.to_csv('amenities_AND_listings.csv', index = False)

## Hosts

### host_response_time: HOST_RESPONSE_TIME_ID; HOST_RESPONSE_TIME

In [157]:
# Those are all the hosts of all the different listings
listingsB_hosts = berlin_listings.iloc[:, 13:23]
listingsM_hosts = madrid_listings.iloc[:, 13:23]
listingsBa_hosts = barcelona_listings.iloc[:, 13:23]
# We concatenate hosts from all cities
listings_hosts = pd.concat([listingsB_hosts, listingsM_hosts, listingsBa_hosts])
# we remove host duplicates since a host can have several listings.
hosts = listings_hosts.drop_duplicates(subset = 'host_id', keep = 'first')

In [163]:
host_resp_time = pd.DataFrame(hosts['host_response_time'].drop_duplicates(keep = 'first')) #filters the host_response_time
host_resp_time['host_response_time_id'] = range(host_resp_time['host_response_time'].count()+1) #creates indexes
host_resp_time = host_resp_time.set_index('host_response_time_id') #set new indexes
host_resp_time.head()

,host_response_time
host_response_time_id,
0,within an hour
1,NaN
2,within a day
3,within a few hours
4,a few days or more


In [164]:
host_resp_time.to_csv('host_response_time.csv')

## hosts

In [166]:
hosts.set_index('host_id') #setting up index
temp = hosts['host_response_rate'].str.replace('%','').values #create new pd.Series of new host_response_rates
hosts = hosts.drop(['host_response_rate'], axis = 1) #remove previous column
hosts['host_response_rate'] = temp #add new column
del(temp)
hosts.head()

,host_id,host_url,host_name,host_since,host_about,host_response_time,host_thumbnail_url,host_picture_url,host_neighbourhood,host_response_rate
0,10692949,https://www.airbnb.com/users/show/10692949,Suites & More,2013-12-18,Like hosts we like to offer what we would like...,within an hour,https://a0.muscache.com/im/users/10692949/prof...,https://a0.muscache.com/im/users/10692949/prof...,Kreuzberg,100
1,10914898,https://www.airbnb.com/users/show/10914898,Anita,2013-12-30,NaN,NaN,https://a0.muscache.com/im/pictures/37a4e6f0-6...,https://a0.muscache.com/im/pictures/37a4e6f0-6...,Friedrichshain,NaN
2,4314868,https://www.airbnb.com/users/show/4314868,Katharina,2012-12-03,Ich lebe in Berlin - im schönen Prenzlauer Ber...,within an hour,https://a0.muscache.com/im/users/4314868/profi...,https://a0.muscache.com/im/users/4314868/profi...,Prenzlauer Berg,90
3,9828152,https://www.airbnb.com/users/show/9828152,Katrin,2013-11-05,"Wir sind leidenschaftliche Berliner, lieben un...",NaN,https://a0.muscache.com/im/users/9828152/profi...,https://a0.muscache.com/im/users/9828152/profi...,Alt-Hohenschönhausen,NaN
4,10962834,https://www.airbnb.com/users/show/10962834,Katharina,2014-01-02,NaN,NaN,https://a0.muscache.com/im/users/10962834/prof...,https://a0.muscache.com/im/users/10962834/prof...,Moabit,NaN


### changing the host_response_time with thier ids

In [167]:
d = host_resp_time.to_dict()['host_response_time']
inv_d = dict([[v,k] for k,v in d.items()])
host_resp_time = hosts['host_response_time'].map(inv_d, na_action = 'ignore')

In [168]:
hosts = hosts.drop(['host_response_time'], axis = 1)
hosts['host_response_time'] = host_resp_time
hosts.head()

,host_id,host_url,host_name,host_since,host_about,host_thumbnail_url,host_picture_url,host_neighbourhood,host_response_rate,host_response_time
0,10692949,https://www.airbnb.com/users/show/10692949,Suites & More,2013-12-18,Like hosts we like to offer what we would like...,https://a0.muscache.com/im/users/10692949/prof...,https://a0.muscache.com/im/users/10692949/prof...,Kreuzberg,100,0
1,10914898,https://www.airbnb.com/users/show/10914898,Anita,2013-12-30,NaN,https://a0.muscache.com/im/pictures/37a4e6f0-6...,https://a0.muscache.com/im/pictures/37a4e6f0-6...,Friedrichshain,NaN,1
2,4314868,https://www.airbnb.com/users/show/4314868,Katharina,2012-12-03,Ich lebe in Berlin - im schönen Prenzlauer Ber...,https://a0.muscache.com/im/users/4314868/profi...,https://a0.muscache.com/im/users/4314868/profi...,Prenzlauer Berg,90,0
3,9828152,https://www.airbnb.com/users/show/9828152,Katrin,2013-11-05,"Wir sind leidenschaftliche Berliner, lieben un...",https://a0.muscache.com/im/users/9828152/profi...,https://a0.muscache.com/im/users/9828152/profi...,Alt-Hohenschönhausen,NaN,1
4,10962834,https://www.airbnb.com/users/show/10962834,Katharina,2014-01-02,NaN,https://a0.muscache.com/im/users/10962834/prof...,https://a0.muscache.com/im/users/10962834/prof...,Moabit,NaN,1


### changing the host_neighborhood by their ids

In [172]:
hosts['host_neighbourhood']

0                             Kreuzberg
1                        Friedrichshain
2                       Prenzlauer Berg
3                  Alt-Hohenschönhausen
4                                Moabit
5                       Prenzlauer Berg
6                             Kreuzberg
7                            Schöneberg
8                        Charlottenburg
9                               Wedding
10                       Friedrichshain
11                            Kreuzberg
12                            Kreuzberg
13                                Mitte
14                             Neukölln
15                                Mitte
16                           Schöneberg
17                      Prenzlauer Berg
18                            Tempelhof
19                      Prenzlauer Berg
20                              Wedding
21                              Wedding
22                             Neukölln
24                               Moabit
25                      Prenzlauer Berg



### Listings locations

In [ ]:
# Those are the geographical locations of the listings
listingsB_locations = berlin_listings.iloc[:, 24:30]
# As for descriptions, there is a one-to-one correspondance between listings and locations
listingsB_locations['listing_id'] = berlin_listings['id']
# note: we could add the city information to the listings so that we could separate the locations in three categories,
#       one for each city, using inheritance
listingsB_locations.head()

### Properties descriptions

In [ ]:
listingsB_properties = berlin_listings.iloc[:, 30:39]
# as usual
listingsB_properties['listing_id'] = berlin_listings['id']
listingsB_properties.head()

### Prices

In [ ]:
listingsB_prices = berlin_listings.iloc[:, 39:48]
# same
listingsB_prices['listing_id'] = berlin_listings['id']
listingsB_prices.head()

### Review scores

In [ ]:
listingsB_reviews = berlin_listings.iloc[:, 48:55]
listingsB_reviews['listing_id'] = berlin_listings['id']
listingsB_reviews.head()

### Permissions

In [ ]:
listingsB_permissions = berlin_listings.iloc[:, 55:]
listingsB_permissions['listing_id'] = berlin_listings['id']
listingsB_permissions.head()

## Reviews

In [ ]:
berlin_reviews.head()

In [ ]:
duplicated_reviewers = berlin_reviews[berlin_reviews.duplicated(['reviewer_id'], keep = False)].count()[0]
# as we can see below, there are many reviewers that made many reviews
reviews = berlin_reviews.count()[0]
print("there are {}% of the reviews that have a reviewer that reviewed at least another listing in Berlin".format(int(duplicated_reviewers/reviews * 100)))

In [ ]:
reviews

In [ ]:
berlin_listings.count()[0]

In [ ]:
reviews_on_diff_listings = berlin_reviews.drop_duplicates(['listing_id'], keep = 'first').sort_values('listing_id').count()[0]
number_of_listings = listingsB.sort_values('id').count()[0]
# note: many listings don't have a review
print("{}% of the listings have a review".format( int(reviews_on_diff_listings / number_of_listings * 100) ))

In [ ]:
listings_ids = listingsB['id']
listings_ids.count()

In [ ]:
reviews_ids = berlin_reviews.drop_duplicates(['listing_id'], keep = 'first')['listing_id']
reviews_ids.count()

In [ ]:
li = listings_ids.sort_values()
ri = reviews_ids.sort_values()
ri[ri.isin(li)].count()
# So every review has a listing id corresponding to a listing

## Calendar

In [ ]:
berlin_calendar[berlin_calendar.notnull()['price']].count()

In [ ]:
id_cal = berlin_calendar[berlin_calendar['listing_id'] == listingsB['id'][3]]
id_cal[first_id_cal['available'] == 't'].count()[0]
# some listings never were available

In [ ]:
# for every listing, verify if the corresponding calendar has at least one true in the list
l = berlin_calendar.drop(['date', 'price'], axis = 1)
available = l[l['available'] == 't'].drop_duplicates(['listing_id'], keep = 'first')
n_av = available.count()[0]
n_lis = listingsB.count()[0]
print("there are {} over {} ({} %) listings that were available at least once in the year"
      .format( str(n_av), str(n_lis), str(int(n_av / n_lis * 100)) ))
# note: we coud easily drop every listings that never was available

In [ ]:
reviews_on_available_number = berlin_reviews[berlin_reviews['listing_id'].isin(available['listing_id'])].count()[0]
reviews_number = berlin_reviews.count()[0]
print("there are {} reviews on available over {} ({} %)"
      .format( str(reviews_on_available_number), str(reviews_number), str(int(reviews_on_available_number / reviews_number * 100))))

In [ ]:
berlin_reviews.info()

In [ ]:
barcelona